<a href="https://colab.research.google.com/github/Byeon-MJ/ML_Practice_Repo/blob/main/Cross_Validation_%26_GridSearchCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 데이터셋

In [24]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [25]:
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [26]:
data = wine.iloc[:,:3].to_numpy()
target = wine['class'].to_numpy()

In [43]:
data

array([[ 9.4 ,  1.9 ,  3.51],
       [ 9.8 ,  2.6 ,  3.2 ],
       [ 9.8 ,  2.3 ,  3.26],
       ...,
       [ 9.4 ,  1.2 ,  2.99],
       [12.8 ,  1.1 ,  3.34],
       [11.8 ,  0.8 ,  3.26]])

### 데이터셋 나누기

* 훈련셋 : 검증셋 : 테스트셋

In [44]:
from sklearn.model_selection import train_test_split
# 훈련셋, 테스트셋 나누기
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [45]:
# 훈련셋, 검증셋 나누기
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [47]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


## 모델링

In [48]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(sub_input, sub_target)
print(dt_clf.score(sub_input, sub_target))
print(dt_clf.score(val_input, val_target))

0.9971133028626413
0.864423076923077


### K_Fold

In [49]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt_clf, train_input, train_target)
print(scores)

{'fit_time': array([0.03466344, 0.01802921, 0.03058624, 0.02216196, 0.01678061]), 'score_time': array([0.00859427, 0.00134468, 0.00119877, 0.00746298, 0.01354599]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [50]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


### StratifiedKFold

In [51]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt_clf, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


* Fold 수 늘리기 ( 10 - Fold)

In [52]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt_clf, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


## 하이퍼파라미터 튜닝


### 그리드 서치

* 각 모델별로 튜닝할수 있는 파라미터가 다르다
* 파라미터는 Sckit-Learn 공식 문서 참조
* 그리드 서치는 CV 기본값이 5

In [53]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [54]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

In [55]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [56]:
# 최적 파라미터 추출
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [57]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [58]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [59]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


#### 그리드서치 과정 정리
1. 탐색할 매개변수 지정
1. 훈련 세트에 그리드 서치를 수행하여 최상의 매개변수 조합 찾기, 그리드 서치 객체에 저장
1. 최상의 매개변수로 전체 훈련 세트를 사용해 모델 훈련

* 복잡한 매개변수 탐색

In [62]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}

In [63]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [64]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [65]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


### 랜덤 서치

* 매개변수 조건이 너무 많으면 그리드 서치로는 시간이 오래 걸릴 수 있음
* 매개변수의 값이 수치일 때, 값의 범위나 간격을 미리 정하기 어려울 경우
* 랜덤 서치를 사용할 수 있다

* randint, uniform 연습

In [66]:
from scipy.stats import uniform, randint

In [69]:
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 4, 4, 9, 1, 6, 3, 6, 8, 7])

In [70]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102,  96,  87, 102, 123,  93,  97,  91, 110,  99]))

In [71]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.65954286, 0.37095204, 0.77174449, 0.16837852, 0.06665385,
       0.70865234, 0.94171474, 0.89841632, 0.08475067, 0.98136317])

In [72]:
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25),}

In [73]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, 
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5be111b1f0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5be111b6a0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5be111ba60>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f5be3984400>},
                   random_state=42)

In [74]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [75]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [76]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
